In [1]:
# import system libs
import os
import time
import shutil
import pathlib
import itertools

# import data handling tools
import cv2
import numpy as np
import pandas as pd
import seaborn as sns
sns.set_style('darkgrid')
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report
from glob import glob

# import Deep learning Libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, Activation, Dropout, BatchNormalization, Conv2DTranspose, Input, Concatenate
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau, CSVLogger
# Ignore Warnings
import warnings
warnings.filterwarnings("ignore")

print ('modules loaded')

modules loaded


In [2]:
# Function to generate data paths with labels
def split_df(csv_dir):
    '''
    This function take csv file and split it into train, valid, and test
    '''

    df = pd.read_csv(csv_dir)

    # train dataframe
    train_df, dummy_df = train_test_split(df,  train_size= 0.7, shuffle= True, random_state= 123)

    # valid and test dataframe
    valid_df, test_df = train_test_split(dummy_df,  train_size= 0.5, shuffle= True, random_state= 123)

    return train_df, valid_df, test_df

In [3]:
def create_gens(train_df, valid_df, test_df, batch_size):
    # define model parameters
    img_size = (224, 224)
    channels = 3
    color = 'rgb'
    img_shape = (img_size[0], img_size[1], channels)
    height = img_size[0]
    width = img_size[1]
    train_dir = 'C:/Users/Yuvraj Chugh/Desktop/Deep Learning/plant-pathology-2021-fgvc8/train_images'
    test_dir = 'C:/Users/Yuvraj Chugh/Desktop/Deep Learning/plant-pathology-2021-fgvc8/test_images'
    fpath_col = 'image'
    label_col = 'labels'
    ts_length = len(test_df)
    test_batch_size = max(sorted([ts_length // n for n in range(1, ts_length + 1) if ts_length%n == 0 and ts_length/n <= 80]))
    test_steps = ts_length // test_batch_size

    def scalar(img):
        return img

    tr_gen = ImageDataGenerator(preprocessing_function= scalar, horizontal_flip= True)
    ts_gen = ImageDataGenerator(preprocessing_function= scalar)

    train_gen = tr_gen.flow_from_dataframe( train_df, directory= train_dir, x_col= fpath_col, y_col= label_col, target_size= img_size,
                                            class_mode= 'categorical', color_mode= color, shuffle= True, batch_size= batch_size)

    valid_gen = ts_gen.flow_from_dataframe( valid_df, directory= train_dir, x_col= fpath_col, y_col= label_col, target_size= img_size,
                                            class_mode= 'categorical', color_mode= color, shuffle= True, batch_size= batch_size)

    test_gen = ts_gen.flow_from_dataframe( test_df, directory= train_dir, x_col= fpath_col, y_col= label_col, target_size= img_size,
                                            class_mode= 'categorical', color_mode= color, shuffle= False, batch_size= test_batch_size)

    return train_gen, valid_gen, test_gen

In [4]:
def show_images(df, data_path):
    sample_df = df.sample(16)
    image_names = sample_df["image"].values
    labels = sample_df["labels"].values
    plt.figure(figsize=(16, 12))
    
    for image_ind, (image_name, label) in enumerate(zip(image_names, labels)):
        plt.subplot(4, 4, image_ind + 1)
        image = cv2.imread(os.path.join(data_path, image_name))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        plt.imshow(image)
        plt.title(f"{label}", fontsize=12)
        plt.axis("off")
    plt.show()

In [5]:
csv_dir = "C:/Users/Yuvraj Chugh/Desktop/Deep Learning/plant-pathology-2021-fgvc8/train.csv"
files_dir = os.path.join("files", "non-aug")
model_file = os.path.join(files_dir, "unet-non-aug.h5")
log_file = os.path.join(files_dir, "log-non-aug.csv")

train_path = 'C:/Users/Yuvraj Chugh/Desktop/Deep Learning/plant-pathology-2021-fgvc8/train_images'
try:
    # Get splitted data
    train_df, valid_df, test_df = split_df(csv_dir)

    # Get Generators
    batch_size = 2500
    train_gen, valid_gen, test_gen = create_gens(train_df, valid_df, test_df, batch_size)

except Exception:
    print('Invalid Input')

Found 13042 validated image filenames belonging to 12 classes.
Found 2795 validated image filenames belonging to 12 classes.
Found 2795 validated image filenames belonging to 12 classes.


In [6]:
# Display Image Samples
# show_images(train_df, train_path)

In [7]:
batch_size=2500
lr= 1e-3
epochs=10
height = 224
width = 224

In [8]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [9]:
create_dir(files_dir)

In [10]:
# Convolution Block

def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding = "same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    
    x = Conv2D(num_filters, 3, padding = "same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x


In [11]:
#Encoder

def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p

In [12]:
#Decoder

def decoder_block(inputs, skip, num_filters):
    x = Conv2DTranspose(num_filters,(2,2), strides=2, padding='same')(inputs)
    x = Concatenate()([x,skip])
    x = conv_block(x,num_filters)
    return x

In [13]:
#UNET
def build_unet(inputs):
    
    """Encode"""
    s1, p1 = encoder_block(inputs,64)
    s2, p2 = encoder_block(p1,128)
    s3, p3 = encoder_block(p2,256)
    s4, p4 = encoder_block(p3,512)
    
    """ Bridge"""
    b1 = conv_block(p4, 1024)
    
    """Decoder"""
    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)
    
    outputs = Conv2D(1,1, padding = 'same')(d4)
    
    flattened_output = Flatten()(outputs)

    # Add three dense layers
    dense1 = Dense(128, activation='relu')(flattened_output)
    dense2 = Dense(64, activation='relu')(dense1)
    dense3 = Dense(32, activation='relu')(dense2)

    # Output layer for 12 categories (adjust the number of units accordingly)
    output = Dense(12, activation='softmax')(dense3)
    
    model = Model(inputs, output, name = "UNET")
    return model

In [14]:
# for x,y in train_gen:
#     print(x.shape, y.shape)

In [15]:
# train_x, train_y = next(train_gen)
# print(train_x.shape)
# print(train_y.shape)

In [16]:
input_shape = (height,width,3)
inputs = Input(shape=input_shape)
model = build_unet(inputs)

In [17]:
model.summary()

Model: "UNET"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 224, 224, 64)         1792      ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 224, 224, 64)         256       ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 224, 224, 64)         0         ['batch_normalization[0][0]

In [18]:
opt = tf.keras.optimizers.Adam(lr)
model.compile(loss="categorical_crossentropy", optimizer = opt, metrics=["acc"])

In [19]:
callbacks = [
    ModelCheckpoint(model_file, verbose=1, save_best_only =True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=4),
    CSVLogger(log_file),
    EarlyStopping(monitor='val_loss', patience = 20, restore_best_weights = False)
]

In [21]:
model.fit(
    train_gen,
    validation_data = valid_gen,
    epochs = epochs,
    callbacks = callbacks
)

Epoch 1/10


ResourceExhaustedError: Graph execution error:

Detected at node UNET/batch_normalization/FusedBatchNormV3 defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "C:\Users\Yuvraj Chugh\anaconda3\Lib\site-packages\ipykernel_launcher.py", line 17, in <module>

  File "C:\Users\Yuvraj Chugh\anaconda3\Lib\site-packages\traitlets\config\application.py", line 992, in launch_instance

  File "C:\Users\Yuvraj Chugh\anaconda3\Lib\site-packages\ipykernel\kernelapp.py", line 736, in start

  File "C:\Users\Yuvraj Chugh\anaconda3\Lib\site-packages\tornado\platform\asyncio.py", line 195, in start

  File "C:\Users\Yuvraj Chugh\anaconda3\Lib\asyncio\base_events.py", line 607, in run_forever

  File "C:\Users\Yuvraj Chugh\anaconda3\Lib\asyncio\base_events.py", line 1922, in _run_once

  File "C:\Users\Yuvraj Chugh\anaconda3\Lib\asyncio\events.py", line 80, in _run

  File "C:\Users\Yuvraj Chugh\anaconda3\Li